In [24]:
import torch
import numpy as np

In [2]:
oracle = [('a', 0.35, 0.46), ('black', 0.46, 0.75), ('and', 0.75, 0.88), ('yellow', 0.88, 1.08), ('bird', 1.08, 1.54), ('with', 1.58, 1.73), ('a', 1.73, 1.79), ('huge', 1.79, 2.43), ('colorful', 2.71, 3.13), ('beak', 3.13, 3.47), ('in', 3.47, 3.66), ('a', 3.66, 3.73), ('cage', 3.73, 4.27)]
jason_word_feature = torch.tensor([[0.5026, 0.8243], [0.8645, 1.0856],[1.2062, 1.3872], [1.5681, 1.7289], [1.8697, 2.1913], [2.3924, 2.4125], [2.7542, 2.8950], [2.9955, 3.1362], [3.1965, 3.4378], [3.5182, 3.6589], [3.7996, 4.1012]])

In [27]:
def half_matching(gt, pred):
    p_gt = 0 # pointer for gt list
    p_pred = 0 # pointer for pred list
    alignment = []
    while p_pred < len(pred):
        gt_s, gt_e = gt[p_gt][1:]
        pred_s, pred_e = pred[p_pred]
        if pred_e < gt_s: # pred_s    pred_e  gt_s    gt_e
            p_pred += 1
        elif pred_s > gt_e: # gt_s    gt_e  pred_s    pred_e 
            p_gt += 1
        elif pred_s <= gt_s and pred_e <= gt_e: # pred_s  gt_s  pred_e    gt_e
            if (pred_e - gt_s) < (gt_s - pred_s): # pred_s     gt_s pred_e    gt_e
                prev_gt_s, prev_gt_e = gt[p_gt-1][1:] # see if the pred seg overlap more with prev gt seg
                if max(prev_gt_e - max(prev_gt_s, pred_s), 0) >= (pred_e - gt_s): #   prev_gt_s  pred_s     prev_gt_e gt_s pred_e    gt_e
                    alignment.append([p_gt-1, p_pred])
                    p_pred += 1
                else:
                    alignment.append([p_gt, p_pred])
                    p_pred += 1              
            else:
                alignment.append([p_gt, p_pred])
                p_pred += 1
        elif pred_s >= gt_s and pred_e <= gt_e:  # gt_s    pred_s  pred_e    gt_e
            alignment.append([p_gt, p_pred])
            p_pred += 1
        elif pred_s >= gt_s and pred_e >= gt_e:  # gt_s   pred_s      gt_e  pred_e
            if (gt_e - pred_s) < (pred_e - gt_e): # gt_s      pred_s  gt_e         pred_e
                p_gt += 1
            else:
                alignment.append([p_gt, p_pred])
                p_pred += 1
        elif pred_s < gt_s and pred_e > gt_e:  # pred_s    gt_s  gt_e       pred_e ||| this shouldn't happen very often
            if (gt_e - gt_s) < (pred_e - pred_s)/2.: # pred_s    gt_s  gt_e       pred_e  
                prev_gt_s, prev_gt_e = gt[p_gt-1][1:]
                next_gt_s, next_gt_e = gt[p_gt+1][1:]
                prev_overlap = max(prev_gt_s - max(prev_gt_s, prev_gt_s), 0)
                next_overlap = max(next_gt_e - max(pred_e, next_gt_s), 0)
                cur_overlap = gt_e - gt_s
                if cur_overlap >= prev_overlap and cur_overlap >= next_overlap:
                    alignment.append([p_gt, p_pred])
                    p_gt += 1
                elif prev_overlap > cur_overlap and prev_overlap >= next_overlap:
                    alignment.append([p_gt-1, p_pred])
                    p_gt +=1
                else:
                    alignment.append([p_gt+1, p_pred])
                    p_pred += 1
            else:
                alignment.append([p_gt, p_pred])
                p_gt += 1
    return np.unique(alignment, axis=0).tolist()



In [30]:
align = half_matching(oracle, jason_word_feature)

In [32]:
for item in align:
    print(oracle[item[0]], jason_word_feature[item[1]])

('black', 0.46, 0.75) tensor([0.5026, 0.8243])
('yellow', 0.88, 1.08) tensor([0.8645, 1.0856])
('bird', 1.08, 1.54) tensor([1.2062, 1.3872])
('with', 1.58, 1.73) tensor([1.5681, 1.7289])
('huge', 1.79, 2.43) tensor([1.8697, 2.1913])
('huge', 1.79, 2.43) tensor([2.3924, 2.4125])
('colorful', 2.71, 3.13) tensor([2.7542, 2.8950])
('colorful', 2.71, 3.13) tensor([2.9955, 3.1362])
('beak', 3.13, 3.47) tensor([3.1965, 3.4378])
('in', 3.47, 3.66) tensor([3.5182, 3.6589])
('cage', 3.73, 4.27) tensor([3.7996, 4.1012])
